In [153]:
import numpy as np
import math
from typing import Tuple
from collections import namedtuple

Hexahedron = namedtuple('Hexahedron', ['ke_lambda', 'fe_lambda', 'ke_mu', 'fe_mu'])

class Homogenization:
    def __init__(self, 
        cell_len_x: int, 
        cell_len_y: int, 
        cell_len_z: int, 
        lambda_: Tuple[float, float], 
        mu: Tuple[float, float], 
        voxel: np.array
    ):
        self.cell_len_x = cell_len_x
        self.cell_len_y = cell_len_y
        self.cell_len_z = cell_len_z

        self.lambda_ = lambda_
        self.mu = mu
        self.voxel = voxel

        # 6 x 6 Constitutive Tensor
        self.constitutuve_tensor = np.zeros(6)

    def homogenize(self):
        ## Initialize
        # Begin discretization
        n_el_x, n_el_y, n_el_z = self.voxel.shape
        n_el = n_el_x * n_el_y * n_el_z
        dx = self.cell_len_x / n_el_x
        dy = self.cell_len_y / n_el_y
        dz = self.cell_len_z / n_el_z

        # The stiffness matrix is broken down into two parts, one for lambda
        # and the other for mu, we'll separately compute them and then
        # combine.
        ke_lambda, fe_lambda, ke_mu, fe_mu = self._compute_hexahedron(
            dx / 2, dy / 2, dz / 2)

        # Applying the kinematic uniform boundary condition for non-periodic 
        # volmes. Here we apply the node numbers and degrees of freedom for 
        # this approximation.
        node_numbers = np.c_[np.arange(1, ((1 + n_el_x) * (1 + n_el_y) * (1 + n_el_z)) + 1)]
        node_numbers = node_numbers.reshape(
            1 + n_el_x, 1 + n_el_y, 1 + n_el_z)
        
        degrees_of_freedom = np.c_[np.arange(1, ((1 + n_el_x) * (1 + n_el_y) * (1 + n_el_z)) + 1)]

        add_x = np.concatenate((np.array([0, 1, 2]), mid, np.array([-3, -2, -1])))
        add_xy = 3 * (n_el_y + 1) * (n_el_x + 1) + add_x

        element_degrees_of_freedom = np.matlib.repmat(
            degrees_of_freedom, 1, 24) + np.matlib.repmat(
                [add_x, add_xy, n_el, 1])

        print(element_degrees_of_freedom)


    def _compute_hexahedron(self, a: int, b: int, c: int) -> Hexahedron:
        """Compute hexahedron from 3d voxel shape grid

        Args:
            a (int): Dimension A (x direction)
            b (int): Dimension B (y direction) 
            c (int): Dimension C (z direction)
        """
        # Constitutive matrix contributions - Mu
        C_mu = np.diag([2, 2, 2, 1, 1, 1])

        # Constitutive matrix contribution - Lambda
        C_lambda = np.zeros((6, 6))
        C_lambda[0:3, 0:3] = 1

        # Calculate the three gauss points in all 3 directions
        xx = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        yy = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        zz = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        ww = np.array([5/9, 8/9, 5/9])

        # Initialize
        ke_lambda = np.zeros((24, 24))
        fe_lambda = np.zeros((24, 6))

        ke_mu = np.zeros((24, 24))
        fe_mu = np.zeros((24, 6))

        # Begin integration over the volume
        for i in range(len(xx)):
            for j in range(len(yy)):
                for k in range(len(zz)):
                    # Integration point
                    x = xx[i]
                    y = yy[j]
                    z = zz[k]

                    # Compute the stress-strain displacement matrix
                qx = [
                    -((y-1)*(z-1))/8, 
                    ((y-1)*(z-1))/8, 
                    -((y+1)*(z-1))/8,
                    ((y+1)*(z-1))/8, 
                    ((y-1)*(z+1))/8,
                    -((y-1)*(z+1))/8,
                    ((y+1)*(z+1))/8, 
                    -((y+1)*(z+1))/8
                ]

                qy = [ 
                    -((x-1)*(z-1))/8,
                    ((x+1)*(z-1))/8,
                    -((x+1)*(z-1))/8,
                    ((x-1)*(z-1))/8,
                    ((x-1)*(z+1))/8, 
                    -((x+1)*(z+1))/8,
                    ((x+1)*(z+1))/8,
                    -((x-1)*(z+1))/8
                ]
                
                qz = [
                    -((x-1)*(y-1))/8,
                    ((x+1)*(y-1))/8, 
                    -((x+1)*(y+1))/8,
                    ((x-1)*(y+1))/8, 
                    ((x-1)*(y-1))/8, 
                    -((x+1)*(y-1))/8,
                    ((x+1)*(y+1))/8, 
                    -((x-1)*(y+1))/8
                ]

                # Compute the jacobian
                J = np.matmul(np.array([qz, qy, qz]), np.array([
                    [-a, a, a, -a, -a, a, a, -a],
                    [-b, -b, b, b, -b, -b, b, b],
                    [-c, -c, -c, -c, c, c, c, c],
                ]).transpose())

                qxyz = np.linalg.lstsq(J, qq)[0]
                B_e = np.zeros((8, 6, 3))

                for ii in range(B_e.shape[0]):
                    B_e[ii] = np.array([
                        [qxyz[0, ii], 0, 0],
                        [0, qxyz[1, ii], 0],
                        [0, 0, qxyz[2, ii]],
                        [qxyz[1, ii], qxyz[0, ii], 0],
                        [0, qxyz[2, ii], qxyz[1, ii]],
                        [qxyz[2, ii], 0, qxyz[0, ii]],
                    ])

                B = np.hstack([B_e[ii] for ii in range(B_e.shape[0])])

                weight = np.linalg.det(J) * ww[i] * ww[j] * ww[k]

                # Element matrices
                ke_lambda += weight * np.matmul(np.matmul(B.transpose(), C_lambda), B)
                ke_mu += weight * np.matmul(np.matmul(B.transpose(), C_mu), B)

                # Element loads
                fe_lambda += weight * np.matmul(B.transpose(), C_lambda)
                fe_mu += weight * np.matmul(B.transpose(), C_mu)
    
        return Hexahedron(ke_lambda, ke_mu, fe_lambda, fe_mu)



In [154]:


h = Homogenization(100, 100, 100, 0, 0, np.zeros((100, 100, 100)))
h.homogenize()


(1030300, 1)


<ipython-input-153-da29d0535694>:141: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  qxyz = np.linalg.lstsq(J, qq)[0]


ValueError: cannot reshape array of size 1030300 into shape (101,101,101)

In [164]:
C = np.zeros((6, 6))
C[0:3, 0:3] = 1
np.linalg.lstsq(C, C)[0]
x = 1
y = 2
z = 3
qx = [
    -((y-1)*(z-1))/8, 
    ((y-1)*(z-1))/8, 
    -((y+1)*(z-1))/8,
    ((y+1)*(z-1))/8, 
    ((y-1)*(z+1))/8,
    -((y-1)*(z+1))/8,
    ((y+1)*(z+1))/8, 
    -((y+1)*(z+1))/8
]

qy = [ 
    -((x-1)*(z-1))/8,
    ((x+1)*(z-1))/8,
    -((x+1)*(z-1))/8,
    ((x-1)*(z-1))/8,
    ((x-1)*(z+1))/8, 
    -((x+1)*(z+1))/8,
    ((x+1)*(z+1))/8,
    -((x-1)*(z+1))/8
]

qz = [
    -((x-1)*(y-1))/8,
    ((x+1)*(y-1))/8, 
    -((x+1)*(y+1))/8,
    ((x-1)*(y+1))/8, 
    ((x-1)*(y-1))/8, 
    -((x+1)*(y-1))/8,
    ((x+1)*(y+1))/8, 
    -((x-1)*(y+1))/8
]

qq = np.array([qx, qy, qz])
J = np.identity(3)
np.linalg.solve(J, qq)

qxyz = np.linalg.solve(J, qq) 
B_e = np.zeros((8, 6, 3))

for i in range(B_e.shape[0]):
    B_e[i] = np.array([
        [qxyz[0, i], 0, 0],
        [0, qxyz[1, i], 0],
        [0, 0, qxyz[2, i]],
        [qxyz[1, i], qxyz[0, i], 0],
        [0, qxyz[2, i], qxyz[1, i]],
        [qxyz[2, i], 0, qxyz[0, i]],
    ])

np.set_printoptions(edgeitems=10, linewidth=100000)
B = np.array([B_e[i] for i in range(B_e.shape[0])])
np.hstack([B_e[i] for i in range(B_e.shape[0])])

n_el_x = 100
n_el_y = 100
n_el_z = 100

print(n_el_x * n_el_y * n_el_z)
node_numbers = np.c_[np.arange(1, ((1 + n_el_x) * (1 + n_el_y) * (1 + n_el_z)) + 1)]
node_numbers = node_numbers.reshape(
    1 + n_el_x, 1 + n_el_y, 1 + n_el_z)

1000000


<ipython-input-164-8ee70230bad6>:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.linalg.lstsq(C, C)[0]
